In [1]:
import os
import numpy as np
import pandas as pd
import argus_shapes

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-03-30 14:49:35,258 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
subject = '12-005'
amplitude = 2.0
electrodes = None
random_state = 42

In [3]:
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')
X, y = argus_shapes.load_data(rootfolder, subject=subject, electrodes=electrodes,
                            amplitude=amplitude, random_state=random_state,
                            verbose=False)
len(X)

110

In [4]:
result = []
for engine in ['cython', 'joblib', 'serial']:
    model = argus_shapes.models.ModelC(engine=engine, n_jobs=-1, xystep=1, axlambda=500)
    model.implant = model.implant_type(x_center=model.implant_x,
                                       y_center=model.implant_y,
                                       rot=model.implant_rot)
    t_fit = %timeit -oq model.fit(X)

    # Run it so we have all the variables we need
    model.build_ganglion_cell_layer()
    bundles = model._grows_axon_bundles()
    axons = model._finds_closest_axons(np.array(bundles))
    
    # time it
    model._curr_map = {}
    t_gcl = %timeit -oq model.build_ganglion_cell_layer()
    t_ofl = %timeit -oq model.build_optic_fiber_layer()
    t_bundles = %timeit -oq model._grows_axon_bundles()
    t_closest = %timeit -oq model._finds_closest_axons(bundles)
    t_contrib = %timeit -oq model._calcs_axon_contribution(axons)
    t_curr_map = %timeit -oq model.calc_curr_map(X)
    
    result.append({'engine': engine,
                   '0_gcl': t_gcl.average,
                   '1_ofl': t_ofl.average,
                   '1a_bundles': t_bundles.average,
                   '1b_closest': t_closest.average,
                   '1c_contrib': t_contrib.average,
                   '2_curr_map': t_curr_map.average,
                   '3_fit': t_fit.average})

In [5]:
pd.DataFrame(result).groupby('engine').mean()

,0_gcl,1_ofl,1a_bundles,1b_closest,1c_contrib,2_curr_map,3_fit
engine,,,,,,,
cython,0.000357,0.043453,0.044127,0.200949,0.027413,0.000071,0.042112
joblib,0.000364,0.151001,0.359074,0.594050,0.132483,0.000072,0.145824
serial,0.000384,0.145933,0.044569,0.584223,0.133071,0.000071,0.147381


In [6]:
%load_ext line_profiler

In [14]:
model.set_params(engine='serial')
model.fit(X)
%lprun -f model.predict model.predict(X)